In [4]:
import pandas as pd
df=pd.read_excel('Data_google_map.xlsx')
df

,Place_index,Place_name,coordinates,Neighbors_indice,"distance(km),time(min)",Neighbors_weight
0,1,TIEC,"35.79105095628615, 51.40298608200715","2,3","(5.3,9),(5.8 ,14 )","14.3,19.8"
1,2,Mellat Park,"35.77823306850121, 51.410100213125865","1,3,4,5","(5.3,9),(4.6,11),(3.5,12),(3.1,7)","14.3,15.6 ,15.5 ,10.1"
2,3,Persian Garden,"35.772070289055634, 51.39034401749359","1,2,5","(5.8 ,14 ),(4.6,11),(4.7,13)","19.8,15.6,17.7"
3,4,Shandiz,"35.771328226614926, 51.42222781465077","2,5,9","(3.5,12),(2.8,9),(5.7,11)","15.5,11.8,16.7"
4,5,Payetakht,"35.76284427056937, 51.41098196750635","2,4,6","(3.1,7),(2.8,9),(4.1,7)","10.1,11.8,11.1"
5,6,Tabiat Bridge,"35.75450272407956, 51.42044824082823","7,10","(6,9),(6.1,12)","15,18.1"
6,7,NMIR,"35.75331515628775, 51.42552061152774","6,8,13","(6,9),(8.5,19),(8.4,13)","15,27.5,21.4"
7,8,Shariati-MS,"35.76274691634308, 51.44422539118962","7,9","(8.5,19),(1.6,6)","27.5,7.6"
8,9,Iranmehr Hospital,"35.76747896011174, 51.44212767701501","4,8,14","(5.7,11),(1.6,6),(11.8,17)","16.7,7.6,28.8"
9,10,TIT,"35.74253787444323, 51.39900388114728","6,11,15","(6.1,12),(6.2,12),(7.4,10)","18.1,18.2,17.4"


# split Neighbors_indice column by comma

In [8]:
df['Neighbors_indice'].str.split(',', expand=True)

,0,1,2,3
0,2,3,None,None
1,1,3,4,5
2,1,2,5,None
3,2,5,9,None
4,2,4,6,None
5,7,10,None,None
6,6,8,13,None
7,7,9,None,None
8,4,8,14,None
9,6,11,15,None


# split Neighbors_weight column by comma

In [22]:
df['Neighbors_weight'].str.split(',', expand=True).astype('float')

,0,1,2,3
0,14.3,19.8,NaN,NaN
1,14.3,15.6,15.5,10.1
2,19.8,15.6,17.7,NaN
3,15.5,11.8,16.7,NaN
4,10.1,11.8,11.1,NaN
5,15.0,18.1,NaN,NaN
6,15.0,27.5,21.4,NaN
7,27.5,7.6,NaN,NaN
8,16.7,7.6,28.8,NaN
9,18.1,18.2,17.4,NaN


# convert Neighbors_weight column to float datatype

In [109]:
def generate_float(col,i):
  a_list =df[col][i].split(',')
  list_of_floats = []
  for item in a_list:
    list_of_floats.append(float(item))
  return list_of_floats

In [110]:
generate_float('Neighbors_weight',0)

[14.3, 19.8]

# write a function to generate nodes and their coresponding edges

In [112]:
def generate_nodes():
  nodes=[]
  for i in range(16):
    nodes.extend(list(zip([str(df['Place_index'][i])]*len(df['Neighbors_weight'][i].split(',')), df['Neighbors_indice'][i].split(','),generate_float('Neighbors_weight',i))))
  return nodes

In [115]:
edges=generate_nodes()
edges

[('1', '2', 14.3),
 ('1', '3', 19.8),
 ('2', '1', 14.3),
 ('2', '3', 15.6),
 ('2', '4', 15.5),
 ('2', '5', 10.1),
 ('3', '1', 19.8),
 ('3', '2', 15.6),
 ('3', '5', 17.7),
 ('4', '2', 15.5),
 ('4', '5', 11.8),
 ('4', '9', 16.7),
 ('5', '2', 10.1),
 ('5', '4', 11.8),
 ('5', '6', 11.1),
 ('6', '7', 15.0),
 ('6', '10', 18.1),
 ('7', '6', 15.0),
 ('7', '8', 27.5),
 ('7', '13', 21.4),
 ('8', '7', 27.5),
 ('8', '9', 7.6),
 ('9', '4', 16.7),
 ('9', '8', 7.6),
 ('9', '14', 28.8),
 ('10', '6', 18.1),
 ('10', '11', 18.2),
 ('10', '15', 17.4),
 ('11', '10', 13.3),
 ('11', '12', 18.2),
 ('11', '16', 28.3),
 ('12', '13', 12.8),
 ('12', '14', 12.8),
 ('13', '7', 21.4),
 ('13', '12', 12.8),
 ('13', '14', 7.7),
 ('13', '15', 11.0),
 ('14', '9', 28.8),
 ('14', '12', 12.8),
 ('14', '13', 7.7),
 ('15', '10', 17.4),
 ('15', '13', 11.0),
 ('15', '16', 6.3),
 ('16', '11', 13.3),
 ('16', '15', 6.3)]

In [149]:
edges[10][:2]

('4', '5')

# define a Graph Class

In [116]:
from collections import defaultdict

class Graph():
    def __init__(self):
        """
        self.edges is a dict of all possible next nodes
        e.g. {'X': ['A', 'B', 'C', 'E'], ...}
        self.weights has all the weights between two nodes,
        with the two nodes as a tuple as the key
        e.g. {('X', 'A'): 7, ('X', 'B'): 2, ...}
        """
        self.edges = defaultdict(list)
        self.weights = {}
    
    def add_edge(self, from_node, to_node, weight):
        # Note: assumes edges are bi-directional
        self.edges[from_node].append(to_node)
        self.edges[to_node].append(from_node)
        self.weights[(from_node, to_node)] = weight
        self.weights[(to_node, from_node)] = weight

In [117]:
graph = Graph()

In [118]:
for edge in edges:
    graph.add_edge(*edge)

# implement dijsktra algorithm

In [119]:
def dijsktra(graph, initial, end):
    # shortest paths is a dict of nodes
    # whose value is a tuple of (previous node, weight)
    shortest_paths = {initial: (None, 0)}
    current_node = initial
    visited = set()
    
    while current_node != end:
        visited.add(current_node)
        destinations = graph.edges[current_node]
        weight_to_current_node = shortest_paths[current_node][1]

        for next_node in destinations:
            weight = graph.weights[(current_node, next_node)] + weight_to_current_node
            if next_node not in shortest_paths:
                shortest_paths[next_node] = (current_node, weight)
            else:
                current_shortest_weight = shortest_paths[next_node][1]
                if current_shortest_weight > weight:
                    shortest_paths[next_node] = (current_node, weight)
        
        next_destinations = {node: shortest_paths[node] for node in shortest_paths if node not in visited}
        if not next_destinations:
            return "Route Not Possible"
        # next node is the destination with the lowest weight
        current_node = min(next_destinations, key=lambda k: next_destinations[k][1])
    
    # Work back through destinations in shortest path
    path = []
    while current_node is not None:
        path.append(current_node)
        next_node = shortest_paths[current_node][0]
        current_node = next_node
    # Reverse path
    path = path[::-1]
    return path

# find the shortest path between two nodes

In [170]:
def return_cost(lst):
  cost=0
  for c in range(len(lst)-1):
    a=lst[c:c+2][0]
    b=lst[c:c+2][1]
    for item in edges:
      if item[:2]==(a,b):
        cost+=item[2]
  return cost

In [171]:
return_cost(['2','5','7'])

10.1

In [178]:
origin='Payetakht'
destination='TMCA'

In [193]:
def find_optimal_rout(origin,destination):
  nodes=dijsktra(graph,df[df['Place_name']==origin]['Place_index'].reset_index()['Place_index'][0].astype('str'),df[df['Place_name']==destination]['Place_index'].reset_index()['Place_index'][0].astype('str'))
  city=[]
  for item in nodes :
    city.append(df['Place_name'].to_numpy()[int(item)-1])
  cost=return_cost(nodes)
  print(f"the origin is {origin} and it's coordinate on map is {df[df['Place_name']==origin]['coordinates'].reset_index()['coordinates'][0]}\n")
  print(f"the destination  is {destination} and it's coordinate on map is {df[df['Place_name']==destination]['coordinates'].reset_index()['coordinates'][0]}\n")
  print(f'The optimal sequence indices between cities are : {nodes}\n')
  print(f'The optimal sequence between cities are : {city}\n')
  print(f'The minimum cost is {cost}')

In [194]:
find_optimal_rout(origin,destination)

the origin is Payetakht and it's coordinate on map is 35.76284427056937, 51.41098196750635

the destination  is TMCA and it's coordinate on map is 35.71128470164337, 51.390615739147286

The optimal sequence indices between cities are : ['5', '6', '7', '13', '14']

The optimal sequence between cities are : ['Payetakht', 'Tabiat Bridge', 'NMIR', 'Mehr Hospital', 'TMCA']

The minimum cost is 55.2
